# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

In [10]:
# Initialize k centers. 2) Compute distances from each point to each center. 3) Assign each point to its nearest 
# center. 4) Recompute centers as cluster means and repeat until convergence.

#### b) Create the builder function for step 1.

In [11]:
build_init_fn <- function(k, seed = NULL) {
  function(X) {
    if (!is.null(seed)) set.seed(seed)
    X <- as.matrix(X)
    X[sample(nrow(X), k), , drop = FALSE]
  }
}


#### c) Create the builder function for step 2.

In [12]:
build_distance_fn <- function() {
  function(X, centers) {
    X <- as.matrix(X); centers <- as.matrix(centers)
    # squared Euclidean distances: ||x||^2 + ||c||^2 - 2 x c^T
    outer(rowSums(X^2), rowSums(centers^2), "+") - 2 * X %*% t(centers)
  }
}


#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [19]:
build_assign_fn_vec <- function() {
  function(distmat) {
    max.col(-distmat, ties.method = "first")
  }
}

#### e) Create the builder function for step 4.

In [20]:
build_update_fn <- function(k) {
  function(X, labels) {
    X <- as.matrix(X)
    centers <- matrix(NA_real_, nrow = k, ncol = ncol(X))
    for (j in seq_len(k)) {
      idx <- labels == j
      centers[j, ] <- if (any(idx)) colMeans(X[idx, , drop = FALSE]) else X[sample(nrow(X), 1), ]
    }
    centers
  }
}


#### f) Combine them all into your own `kmeans` function.

In [21]:
kmeans_mine <- function(X, k, max_iter = 100, tol = 1e-6, seed = NULL, use_vectorized = TRUE) {
  init_fn <- build_init_fn(k, seed)
  dist_fn <- build_distance_fn()
  assign_fn <- if (use_vectorized) build_assign_fn_vec() else build_assign_fn_loop()
  update_fn <- build_update_fn(k)

  centers <- init_fn(X)
  for (it in seq_len(max_iter)) {
    distmat <- dist_fn(X, centers)
    labels  <- assign_fn(distmat)
    new_centers <- update_fn(X, labels)
    if (max(abs(new_centers - centers)) < tol) break
    centers <- new_centers
  }
  list(labels = labels, means = centers, iter = it)
}


## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set. 

In [22]:
voltages_df <- read.csv("voltages_df.csv", check.names = FALSE)


#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`. 

In [23]:
set.seed(42)
res_mine <- kmeans_mine(voltages_df, k = 3, max_iter = 100, tol = 1e-6, seed = 42)
print(res_mine$labels)
print(res_mine$means)


  [1] 2 3 3 3 3 1 2 2 3 2 2 2 2 2 3 2 2 2 2 2 1 3 3 3 3 2 2 2 2 2 2 2 3 2 2 3 3
 [38] 2 2 1 2 2 2 2 1 2 2 3 2 2 1 2 2 2 2 2 1 2 2 3 2 2 3 2 2 2 1 2 2 2 2 3 2 2
 [75] 3 1 2 2 2 2 2 2 3 2 2 3 3 3 2 2 2 2 3 2 3 1 2 3 3 2 2 2 3 2 2 2 2 2 2 1 2
[112] 2 1 2 2 2 2 2 3 3 1 3 2 2 2 1 3 2 2 2 3 2 3 2 2 1 3 1 2 2 2 3 3 2 2 2 3 2
[149] 2 2 2 2 3 3 2 2 2 2 1 2 2 2 2 3 2 2 3 2 3 2 2 2 3 2 2 2 3 2 2 1 2 3 2 2 2
[186] 1 1 2 1 2 2 1 2 2 2 2 2 1 2 2 2 2 2 2 3 2 3 2 1 2 1 2 2 2 2 3 2 2 3 3 3 1
[223] 3 3 1 2 2 2 2 2 3 2 2 3 3 2 2 2 2 2 2 2 2 2 3 2 2 2 2 1 2 3 2 2 1 2 2 2 2
[260] 2 3 3 2 2 3 3 3 2 2 2 2 1 1 1 2 2 2 2 1 3 2 2 2 2 2 1 2 3 3 2 2 2 2 2 2 1
[297] 2 2 2 3 3 2 2 2 2 2 2 1 3 1 2 2 2 2 2 2 3 2 3 2 2 2 3 2 2 2 2 2 3 2 1 2 2
[334] 2 1 3 2 2 2 3 2 3 2 2 2 2 3 2 2 2 2 3 1 3 2 2 3 2 2 2 1 3 3 3 2 2 2 1 2 2
[371] 2 2 2 3 1 2 2 2 2 2 2 2 2 2 2 2 2 1 3 2 2 3 3 1 3 2 1 3 2 2 2 2 2 3 3 2 2
[408] 2 2 2 3 2 2 2 3 1 1 2 3 3 3 2 2 1 2 2 3 1 3 1 2 2 1 2 2 1 2 2 2 2 2 2 2 2
[445] 3 2 2 2 2 3 2 2 2 1 3 2 3 3 2 2 3 

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`. 
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [24]:
set.seed(42)
X <- as.matrix(voltages_df)
res_r <- kmeans(X, centers = 3)
print(res_r$centers)   # equivalent to res_mine$means
print(res_r$cluster)   # equivalent to res_mine$labels


          0 1.00401606425703 2.00803212851406 3.01204819277108 4.01606425702811
1 -1.031463         1.244743        1.0934150        0.9018995        0.3150252
2 -1.031463         1.123724        0.9618318        0.6709520       -0.2348958
3 -1.031463         1.243631        1.0920450        0.8997900        0.2949530
  5.02008032128514 6.02409638554217 7.0281124497992 8.03212851405623
1        -1.160098        -1.110416       -1.069455       -1.0351390
2        -1.109877        -1.048146       -0.964286       -0.8417775
3        -1.159542        -1.109542       -1.068141       -1.0332925
  9.03614457831325 10.0401606425703 11.0441767068273 12.0481927710843
1       -1.0039365       -0.9721446      -0.93706913      -0.89765129
2       -0.7449379       -0.5477344      -0.07723852       0.03795671
3       -1.0014773       -0.9689990      -0.93315694      -0.89288201
  13.0522088353414 14.0562248995984 15.0602409638554 16.0642570281125
1       -0.8546429       -0.8106098       -0.7699049  

#### d) Are your labels/clusters the same? If not, why? Are your means the same?

In [25]:
# Labels need not match because cluster IDs are arbitrary and starts differ. Means often differ if algorithms 
# or initial centers differ. If you fix the same initial centers and use Lloyd updates, centers match up to a 
# permutation of labels.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

In [26]:
# For each point, compute its distance to each center, pick the index of the smallest distance, store that index as the label.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

In [27]:
# Compute the full distance matrix for all points versus all centers in one call, then take max.col(-distmat) to get the nearest center per row.

#### c) State which (for loops or vectorizing) is more efficient and why.

In [28]:
# Vectorization is faster because it uses optimized BLAS matrix operations and reduces R interpreter overhead.

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

In [29]:
# It fails with non‑spherical, unequal‑size, unequal‑density, or non‑convex clusters, and with strong outliers. It assumes clusters are roughly spherical and separable by Euclidean distance with similar variance.

## Question 5
#### What assumption do Guassian mixture models make?

In [30]:
# Data are generated from a finite mixture of Gaussian distributions with component‑specific means and covariances and fixed mixing weights. Points are independent given the mixture.

## Question 6
#### What assumption does spectral clustering make? Why does this help us?

In [31]:
# Assumes points within a cluster have high graph similarity and clusters are weakly connected to each other. The Laplacian embedding separates such groups so simple methods like k‑means can partition non‑convex shapes.

## Question 7
#### Define the gap statistic method. What do we use it for?

In [32]:
# Compares log within‑cluster dispersion for k to its expectation under a reference null distribution. Select k that maximizes the gap, often using a one‑standard‑error rule.